## Classes and functions (wrapper)

In [58]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"


from TDSE_solve import TDSE
from Slater_State import Slater
from Molecule import get_molecule

from random import randint, seed

class Build_basis:
   '''
   calls the necessary classes and functions from SLater_States, TD_pulse and Molecule to
   - build a basis of Slater States
   - Solve the TDSE with given laser pulse(s)
   - plot the expansion coefficients
   '''

   def __init__(self, molecule_name="CO", basis=0):

      # Choose molecular system
      self.mol,self.mf = get_molecule(molecule_name,0)

      # build Slater states
      S_State = Slater(self.mol, self.mf)  # slater state object
      S_State.build()

      self.det = S_State.det  # Slater determinants
      self.dim = len(self.det)
      self.Ene = S_State.Ene  # energie of the Slater states
      self.DE = S_State.DE  # list of Delta(E)
      self.S_tdm = S_State.S_tdm  # transition dipole elements
      self.Orb_ex_ind = S_State.Orb_ex_ind # index of orbital transition wrt the GS

      self.LUMO_idx = np.argmax(np.where(S_State.mo_ene < 0, S_State.mo_ene, 0))
      self.HOMO_idx = self.LUMO_idx-1

      self.Result = None 
      self.ct_list = None
      self.t_list = None
      self.V_list = None 
      self.laser = None
        
   def Solve_TDSE(self,laser,tf,dt=0.005):
    
      # Expansion coefficients, initial condition
      c0 = np.zeros(self.dim,dtype=complex)
      c0[0] = complex(1.0,1.0)

      # Solve TDSE
      my_tdse = TDSE(self.S_tdm,self.DE,laser=laser)
      Result = my_tdse.Solve(c0, tf,dt=dt)
      self.ct_list, self.t_list, self.V_list = Result
      self.laser = laser
     
   def _Plot(self,thres):
      '''
      plot using pyplot
      Depreciated -> use plotly function plot
      '''
        
      # colors
      seed(6) # for nice colors
      colors = []
      for i in range(self.dim):
         colors.append('#%06X' % randint(0, 0xFFFFFF))
         color_field = ('black','red','blue')

      # initialize figure
      fig = plt.figure()
      ax = fig.add_axes([0.1, 0.1, 1, 1.15])
      ax.set_xlabel("time (au)")
      ax.set_ylabel("$|c_i(t)|^2$")
      # plot each probability amplitude (|c^2|) as a function of time
      for i in range(self.dim):
         if np.max(self.ct_list[:,i]) > thres:
            state = f"{self.Orb_ex_ind[i][0]}->{self.Orb_ex_ind[i][1]}"
            ax.plot(self.t_list,self.ct_list[:,i],label=state,color=colors[i])

      # add subplot for field      
      inset = fig.add_axes([0.8, 0.5, 0.25, 0.30]) # left, bottom, width, height
      inset.set_title("Field")
      inset.set_xlabel("time (au)")

      for i in range(self.V_list[0,:].shape[0]):
         inset.plot(self.t_list,self.V_list[:,i],color=color_field[i])

      ax.legend(bbox_to_anchor=(1.05, 1), ncol=6, shadow=True, title="Legend", fancybox=True,loc=2, borderaxespad=0.)
      
      return fig
   
   def Plot(self,thres):
      '''
      Use plotly to plot the expansion coefficients as a function of time
      '''
        
      # colors
      seed(6) # for nice colors
      colors = []
      for i in range(self.dim):
         colors.append('#%06X' % randint(0, 0xFFFFFF))
         color_field = ('black','red','blue')

      # initialize figure
      #fig = go.Figure()
      fig = make_subplots(
         rows=2, cols=2,
         specs=[[{"rowspan": 2},None],
               [None,{}]],
         print_grid=False)

      # plot each probability amplitude (|c^2|) as a function of time
      for i in range(self.dim):
         if np.max(self.ct_list[:,i]) > thres:
            # GS coefficient
            if (self.Orb_ex_ind[i][0] == 0 ) & (self.Orb_ex_ind[i][1] == 0):
               fig.add_trace(go.Scatter(x=self.t_list, y=self.ct_list[:,i],
                    mode='lines',
                    name=f"{self.Orb_ex_ind[i][0]}->{self.Orb_ex_ind[i][1]}",
                    line=dict(color=colors[i], width=2, dash='dash')),
                    row=1, col=1)
            else:
               fig.add_trace(go.Scatter(x=self.t_list, y=self.ct_list[:,i],
                    mode='lines',
                    name=f"{self.Orb_ex_ind[i][0]}->{self.Orb_ex_ind[i][1]}",
                    line=dict(color=colors[i], width=2)),
                    row=1, col=1)   
      
      # add labels and titles
      fig.update_layout(
      title="Expansion coefficient as a function of time",
      xaxis_title="time (au)",
      yaxis_title=r"$|c_i(t)|^2$",
      legend=dict(
         orientation='h',
         yanchor="top",
         y=-0.2,
         #xanchor="left",
         #x=0.8,
         font=dict(size= 12)
         )
      )

      # add subplot for field      
      for i in range(self.V_list[0,:].shape[0]):
         fig.add_trace(go.Scatter(x=self.t_list, y=self.V_list[:,i],
                    mode='lines',
                    showlegend=False,
                    line=dict(color=color_field[i], width=2)),
                    row=2, col=2)
         fig.add_annotation(x=self.laser['t0'][i], y=self.laser["E0"][i],
            text="{:.2f} au".format(abs(self.laser['freq'][i])),
            font=dict(color=color_field[i]),
            row=2, col=2)
      
      return fig
        

## Analysis

In [60]:
# Initialize molecule (CO by default)
my_molecular_system = Build_basis('h2o')

In [50]:
# print some informations

print("HOMO index = ",my_molecular_system.HOMO_idx)
print("LUMO index = ",my_molecular_system.LUMO_idx)

print("Number of basis (Slater state) is ", my_molecular_system.S_tdm.shape[0])
print()
print("GS -> 1st excited states transition")
idx_1st = np.argmin(my_molecular_system.DE[0,1:])+1
print("Energy = {:.2f} eV".format(my_molecular_system.DE[0,idx_1st]*27.2114))
print("TDM = ", my_molecular_system.S_tdm[0,idx_1st])
print("Orb excitation : ", my_molecular_system.Orb_ex_ind[idx_1st])
print()
print("GS -> last excited states transition")
idx_last = np.argmax(my_molecular_system.DE[0,:])
print("Energy = {:.2f} eV".format(my_molecular_system.DE[0,idx_last]*27.214))
print("TDM = ", my_molecular_system.S_tdm[0,idx_last])
print("Orb excitation : ", my_molecular_system.Orb_ex_ind[idx_last])
print()


HOMO index =  4
LUMO index =  5
Number of basis (Slater state) is  41

GS -> 1st excited states transition
Energy = 15.61 eV
TDM =  [ 0.12919609 -0.01666006 -0.0271613 ]
Orb excitation :  [array([4]), array([5])]

GS -> last excited states transition
Energy = 618.52 eV
TDM =  [-0.00254829  0.00137527 -0.01296482]
Orb excitation :  [array([0]), array([12])]



In [61]:
# transition frequencies for GS -> state 2 (core) -> state 26 (valence) 
d1 = my_molecular_system.DE[0,3]   # core excitation
d2 = my_molecular_system.DE[0,26]  # valence

print()
print("Laser energies corresponding to the ")
print("Core state charecterized by ", my_molecular_system.Orb_ex_ind[3] ," transition")
print("Valence state charecterized by ", my_molecular_system.Orb_ex_ind[26] ," transition")
print()

# Choose laser pulse

# Laser pulse parameters in a.u
E0 = [2, 0.5] # intensities
tau = [2, 2] #width -> 1/DE ?
freq = [d1, d2]  # energies of the field
t0 = [3, 6] # time at max(E0)
# integration parameters
tf = 10 # final time
laser = {'E0':E0,'tau':tau,'freq':freq,'t0':t0}

# Solve TDSE
my_molecular_system.Solve_TDSE(laser,tf)

# Plot results
#plt.rcParams["figure.figsize"]=8,6
f = my_molecular_system.Plot(0.01)
f.show()


Laser energies corresponding to the 
Core state charecterized by  [array([0]), array([7])]  transition
Valence state charecterized by  [array([3]), array([6])]  transition

There are 2000.0 integration points
